In [6]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import re
import urllib.request

from dateutil.parser import parse
from urllib.request import urlopen
import ssl

In [7]:
context = ssl._create_unverified_context()
titles = []
date = []
links = []
# li_tag = []
not_good = ['종료', '완료', '마감']

page_num = 0
# 24 48 72
while(page_num<5):
    url = 'https://okky.kr/articles/gathering?offset='+str((page_num)*24)+'&max=24&sort=id&order=desc'
    req = urlopen(url, context=context)
    res = req.read()
    soup = BeautifulSoup(res,'html.parser')
    for i in range(4, 28):
        title = soup.select('.list-title-wrapper.clearfix > h5 > a')[i].text.replace('\n', '').lstrip().rstrip()
        dd = soup.select('div.date-created > span')[i].get_text()
        link = soup.select('.list-title-wrapper.clearfix > h5 > a')[i]['href']
        if (any(word in title for word in not_good) or len(title) < 3):
            continue
        else:
            titles.append(title)
            links.append('https://okky.kr'+link)
            day, write_time = dd.split(" ")
            date.append(day)

            # 태그를 추출해 추가하기에는 페이지에서 보여지는 태그가 애매하다.
            # 따로 키워드를 선정하거나 할 필요가 있어보임
            '''
            for j in range(1, 8):
                tmp = []
                try:
                    tag = soup.select('#list-article > div.panel.panel-default.gathering-panel > ul > li:nth-child('+str(i-3)+') > div.list-title-wrapper.clearfix > div > a')[j].get_text()
                    tmp.append(tag)
                except:
                    continue
                li_tag.append(tmp)
            '''
       
           
    page_num+=1

In [8]:
# 반응이 나쁜 게시물 제거
for tmp in links:
    url = tmp
    req = urlopen(url, context=context)
    res = req.read()
    soup = BeautifulSoup(res,'html.parser')
    bad = int(soup.select('.content-eval-count')[0].text)
    if bad < 0:
        num = links.index(tmp)
        del titles[num]
        del links[num]
        del date[num]
    else: continue


In [9]:
okky = []

for i in range(len(titles)):
    li_tmp = {"title": titles[i], "d-day": date[i], "link": links[i]}
    okky.append(li_tmp)

df = pd.DataFrame(okky)
df

,title,d-day,link
0,프론트엔드 공부 같이 하실 분 모십니다(javasript 30),2022-05-12,https://okky.kr/article/1225202
1,"[사이드 프로젝트] 디자이너 1, 프론트엔드 1 모집합니다!",2022-05-12,https://okky.kr/article/1224903
2,"블록체인 스마트컨트랙트,솔리디티 같이 스터디하실분",2022-05-11,https://okky.kr/article/1224853
3,한양대학교 에리카 모각코,2022-05-11,https://okky.kr/article/1224778
4,[취준 병행 포트폴리오] 스프링 백엔드 두분 구합니다.,2022-05-11,https://okky.kr/article/1224771
...,...,...,...
84,"(독학)취준생과 이직자를 위한 온라인 모각코 하드 스터디 9시~ 6시 (캠 O, 화...",2022-04-27,https://okky.kr/article/1213988
85,WASM(웹어셈블리) 모임/스터디 어셈넥스트 팀,2022-04-27,https://okky.kr/article/1213945
86,개발자 중심 영어 스터디(토요일 9:30AM),2022-04-27,https://okky.kr/article/1213451
87,[신천역][부천역] flutter 스터디or 코테 스터디 모집합니다.,2022-04-27,https://okky.kr/article/1213415


In [10]:
import json

with open('okky.json', 'w', encoding="utf-8") as make_file: 
    json.dump(okky, make_file, ensure_ascii = False, indent="\t")